-----
# cartoGRAPHs 

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

Please note: 
Large graphs (e.g. 20k nodes / 300k links) can take ~15min using TSNE-based layouts and ~5min using UMAP-based layouts

-----

In [2]:
from cartoGRAPHs import *

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import * 
from func_exportVR import * 

_____
# 1 | DEFINE NETWORK
_____

In [3]:
# ---------------------------------------------------------------------------
# 
# this Graph is an exemplary graph and can be replaced by any networkx Graph
#
# ---------------------------------------------------------------------------

n = 1093
r = 3
G = nx.full_rary_tree(r,n)

# ---------------------------------------------------------------------------
#
# this will be hover-info of interactive layouts and can be replaced by any list with length G.nodes
#
# ---------------------------------------------------------------------------

l_features = ['nodeID:'+str(i) for i in list(G.nodes())]

_____ 
# 2 | SPATIAL EMBEDDING
_____

Layouts to choose from: 
+ local
+ global / node2vec
+ importance / struc2vec
+ functional


#### LOCAL 

2D

In [3]:
layoutmethod = 'local'

posG_local_tsne2D = layout_local_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_local_umap2D = layout_local_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

3D

In [5]:
layoutmethod = 'local'

posG_local_tsne3D = layout_local_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_local_umap3D = layout_local_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

#### GLOBAL 

2D

In [6]:
layoutmethod = 'global'

posG_global_tsne2D = layout_global_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_global_umap2D = layout_global_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

3D

In [7]:
layoutmethod = 'global'

posG_global_tsne3D = layout_global_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_global_umap3D = layout_global_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

#### IMPORTANCE 

2D

In [10]:
layoutmethod = 'importance'

posG_importance_tsne2D = layout_importance_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_importance_umap2D = layout_importance_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

3D

In [11]:
layoutmethod = 'importance'

posG_importance_tsne3D = layout_importance_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_importance_umap3D = layout_importance_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

#### FUNCTIONAL

In [42]:
scale = 1
val = 0
rows = len(list(G.nodes()))

feat_one = [(val) if i%3 else (scale) for i in range(rows)]
feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]

feat_matrix = np.vstack((feat_one,feat_two,feat_three))
FM = pd.DataFrame(feat_matrix)
FM.index = ['100','101','102']
FM=FM.T
FM.index = list(G.nodes())

2D

In [14]:
layoutmethod = 'functional'

posG_functional_tsne2D = layout_functional_tsne(G, FM,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_functional_umap2D = layout_functional_umap(G, FM,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

3D

In [15]:
layoutmethod = 'functional'

posG_functional_tsne3D = layout_functional_tsne(G, FM,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_functional_umap3D = layout_functional_umap(G, FM,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

#### NODE2VEC (stellargraph)
+ resource: https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/node-classification/node2vec-weighted-node-classification.ipynb#scrollTo=2RX_cXaLBiQZ

2D

In [29]:
layoutmethod = 'nodevec'

posG_nodevec_tsne2D = layout_nodevec_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_nodevec_umap2D = layout_nodevec_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

3D

In [9]:
layoutmethod = 'nodevec'

posG_nodevec_tsne3D = layout_nodevec_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_nodevec_umap3D = layout_nodevec_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 

#### ATTRI2VEC (stellargraph)
+ resource: https://stellargraph.readthedocs.io/en/v1.0.0/demos/embeddings/attri2vec-embeddings.html

In [56]:
scale = 1
val = 0
rows = len(list(G.nodes()))

feat_one = [(val) if i%3 else (scale) for i in range(rows)]
feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]

feat_matrix = np.vstack((feat_one,feat_two,feat_three))
FM = pd.DataFrame(feat_matrix)
FM.index = ['100','101','102']
FM=FM.T
FM.index = list(G.nodes())

2D

In [55]:
layoutmethod = 'attrivec'

posG_attrivec_tsne2D = layout_attrivec_tsne(G,FM,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_attrivec_umap2D = layout_attrivec_umap(G,FM,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

3D

In [ ]:
layoutmethod = 'attrivec'

posG_attrivec_tsne3D = layout_attrivec_tsne(G,FM,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_attrivec_umap3D = layout_attrivec_umap(G,FM,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

#### GRAPHWAVE (stellargraph)
+ resource: https://stellargraph.readthedocs.io/en/v1.0.0/demos/embeddings/graphwave-embeddings.html

2D

In [81]:
layoutmethod = 'graphwave'

posG_graphwave_tsne2D = layout_graphwave_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_graphwave_umap2D = layout_graphwave_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

3D

In [ ]:
layoutmethod = 'graphwave'

posG_graphwave_tsne3D = layout_graphwave_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_graphwave_umap3D = layout_graphwave_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

#### METAPATH2VEC (stellargraph)

2D

In [52]:
layoutmethod = 'metapathvec'

posG_metapathvec_tsne2D = layout_metapathvec_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_metapathvec_umap2D = layout_metapathvec_umap(G,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

3D

In [11]:
layoutmethod = 'metapathvec'

posG_metapathvec_tsne3D = layout_metapathvec_tsne(G,3,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_metapathvec_umap3D = layout_metapathvec_umap(G,3,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine')

____________
# 3 | SET VISUAL PRE-SETTINGS
____________
+ colors in hex: https://htmlcolorcodes.com/

#### NODES - GENERAL

In [46]:
opacity_nodes = 0.8
node_edge_col = '#696969' 

scale_factor = 0.75
size = list(draw_node_degree(G, scale_factor).values())

scale_factor3D = 1.5
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

In [47]:
# ---------------------------------------------------------------------------
#
# Examplary Node color Parameter: 
# (can be replaced with any dict with keys = G.nodes and values (int or float) > each value will be represented by one color (function: color_nodes_from_dict()) 
#
# ---------------------------------------------------------------------------

closeness = nx.closeness_centrality(G)
d_node_colors = {}
for node, cl in sorted(closeness.items(), key = lambda x: x[1], reverse = 1):
    d_node_colors[node] = round(cl,4)
        
col_pal = 'YlOrRd'
d_colors = color_nodes_from_dict(G, d_node_colors, palette = col_pal)
colors = list(d_colors.values())
edge_color = '#d3d3d3'

#### EDGES - GENERAL

In [48]:
edge_width = 0.1

edge_colorlight = '#d3d3d3' # 'lightgrey'
edge_colordark = '#696969' 
edge_color = '#ACACAC'

opacity_edges = 0.1 

_____ 
# 4 | VISUALIZING
_____
Layout Maps to choose from: 
+ Portrait 2D 
+ Portrait 3D 
+ Topographic Map
+ Geodesic Map 

### 2D PORTRAIT

In [53]:
# ---------------------------------------------------------------------------
#
# SELECT PARAMETER 
#
posG = posG_metapathvec_umap2D
# ---------------------------------------------------------------------------

umap_nodes = get_trace_nodes_2D(posG, l_features, colors, size, opacity_nodes)
umap_edges = get_trace_edges_2D(G, posG, edge_color, opac = 0.5)
data = [umap_nodes,umap_edges]

path = 'output_plots/2Dlayouts/'
fname = '2Dportrait'+'_'+layoutmethod

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_metapathvec.html'

#### EXPORT FOR VRNetzer 

In [38]:
VRpath = 'output_plots/VRlayouts/'
layout_namespace = fname

export_to_csv2D(VRpath, layout_namespace, posG, colors)

### 3D PORTRAIT 

In [18]:
# ---------------------------------------------------------------------------
#
# SELECT PARAMETER
#
posG = posG_functional_umap3D
# ---------------------------------------------------------------------------

umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size3d, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.5, linewidth = 0.5)
data = [umap_nodes,umap_edges]

path = 'output_plots/3Dlayouts/'
fname = '3Dportrait'

plot_3D(data,path,fname, 'light')

'output_plots/3Dlayouts/3Dportrait.html'

#### EXPORT FOR VRNetzer 

In [13]:
VRpath = 'output_plots/VRlayouts/'
layout_namespace = fname

export_to_csv3D(VRpath, layout_namespace, posG, colors)

### TOPOGRAPHIC MAP

In [14]:
# ---------------------------------------------------------------------------
# SELECT a z-Parameter: 
#
# d_z > dictionary with keys=G.nodes and values=any int/float assigned to a node
# 
# ---------------------------------------------------------------------------
#a random example: 
z_list = [np.random.random() for i in range(0, len(list(G.nodes())))]
d_z = dict(zip(list(G.nodes()),z_list))
#
# ---------------------------------------------------------------------------
#
# requirement: 2D Portrait 
# example: 
posG_local_tsne2D = layout_local_tsne(G,2,prplxty=10, density=1, l_rate=200, steps=250, metric='cosine') 
posG_topographic = layout_topographic(posG_local_tsne2D, d_z)
posG = posG_topographic
# ---------------------------------------------------------------------------

umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size3d, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.5, linewidth = 0.5)
data = [umap_nodes,umap_edges]

path = 'output_plots/Topographic/'
fname = 'topographic_map'

plot_3D(data,path,fname, 'light')

'output_plots/Topographic/topographic_map.html'

#### EXPORT FOR VRNetzer 

In [15]:
VRpath = 'output_plots/VRlayouts/'
layout_namespace = fname

export_to_csv3D(VRpath, layout_namespace, posG, colors)

### GEODESIC MAP

In [11]:
# ---------------------------------------------------------------------------
#
# SELECT a r-Parameter:
#
# d_rad > a dictionary with keys=G.nodes and values=any radius assigned to each node
# ---------------------------------------------------------------------------
# example: 
rad_list = [np.random.randint(1,4) for i in range(0, len(list(G.nodes())))]
d_rad = dict(zip(list(G.nodes()), rad_list))
# ---------------------------------------------------------------------------

posG_sphere = layout_geodesic(G, d_rad, n_neighbors=20, spread=1, min_dist=0.0)
posG = posG_sphere

umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size3d, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.5, linewidth = 0.5)
data = [umap_nodes,umap_edges]

path = 'output_plots/Geodesic/'
fname = 'geodesic_map'
plot_3D(data,path,fname, 'light')

'output_plots/Geodesic/geodesic_map.html'

#### EXPORT FOR VRNetzer 

In [12]:
VRpath = 'output_plots/VRlayouts/'
layout_namespace = fname

export_to_csv3D(VRpath, layout_namespace, posG, colors)